<a href="https://colab.research.google.com/github/IgnasiOliveras/anonimitzar/blob/main/Anonimitzacio_versiodefinitiva.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import random
import re
import spacy

!python -m spacy download es_core_news_sm

# Cargar modelo de spaCy en español
nlp = spacy.load("es_core_news_sm")

# Función de preprocesamiento
def preprocesar_texto(texto):
    """Convierte el texto a minúsculas y elimina signos de puntuación."""
    texto = texto.lower()
    texto = re.sub(r'[^\w\sÁÉÍÓÚáéíóúÑñ]', '', texto)
    return texto

# Cargar datos desde el archivo Excel
df = pd.read_excel("MOSTRA_1.xlsx")

# Aplicar preprocesamiento
df["body_preprocesado"] = df["body"].astype(str).apply(preprocesar_texto)

# Guardar preprocesado
df.to_excel("MOSTRA_1_preprocesado.xlsx", index=False)

# Cargar lista de nombres
nombres_df = pd.read_csv("2024_pad_m_nom_sexe.csv")
nombres_masculinos = set(nombres_df[nombres_df["SEXE"] == 2]["NOM"].str.strip().str.lower().tolist())
nombres_femeninos = set(nombres_df[nombres_df["SEXE"] == 1]["NOM"].str.strip().str.lower().tolist())

# Palabras que no se deben anonimizar
palabras_excluidas = {"bo", "d'una", "dan", "justo", "salud", "li", "leo", "te", "ella", "el", "los", "las",
                      "nosotros", "vosotros", "usted", "ustedes", "ellos", "ellas", "su", "sus", "un", "una", "unos",
                      "unas", "nada", "mira", "duna", "rabia", "dora", "saba", "esperanza", "domingo", "sol"}

# Diccionario para consistencia en anonimización
nombre_map = {}
cambios = []

# Funciones auxiliares
def detectar_genero(nombre):
    nombre = nombre.lower()
    if nombre in nombres_masculinos:
        return "male"
    elif nombre in nombres_femeninos:
        return "female"
    return "neutral"

def mantener_formato(original, nuevo):
    if original.istitle():
        return nuevo.title()
    elif original.isupper():
        return nuevo.upper()
    return nuevo.lower()

def generar_nombre(original_name):
    palabras = original_name.split()
    num_palabras = len(palabras)

    if original_name in nombre_map:
        return nombre_map[original_name]

    genero = detectar_genero(palabras[0])
    nuevo_nombre = []

    for _ in range(num_palabras):
        if genero == "male":
            nuevo_nombre.append(random.choice(list(nombres_masculinos)))
        else:
            nuevo_nombre.append(random.choice(list(nombres_femeninos)))

    nombre_final = " ".join(nuevo_nombre)
    nombre_final = mantener_formato(original_name, nombre_final)

    nombre_map[original_name] = nombre_final
    return nombre_final

def anonimizar_texto(texto):
    texto = str(texto)
    palabras = texto.split()
    texto_anonimizado = []

    for i, palabra in enumerate(palabras):
        palabra_limpia = re.sub(r'[^A-Za-zÁÉÍÓÚáéíóúÑñ]', '', palabra).lower()
        if palabra_limpia in nombres_masculinos or palabra_limpia in nombres_femeninos:
            if palabra_limpia not in palabras_excluidas:
                nombre_anonimizado = generar_nombre(palabra)
                texto_anonimizado.append(nombre_anonimizado)

                palabra_anterior = " ".join(palabras[i-3:i]) if i >= 3 else " ".join(palabras[:i])
                palabra_posterior = " ".join(palabras[i+1:i+4]) if i < len(palabras) - 3 else " ".join(palabras[i+1:])

                cambios.append((palabra, nombre_anonimizado, palabra_anterior, palabra_posterior))
            else:
                texto_anonimizado.append(palabra)
        else:
            texto_anonimizado.append(palabra)

    return " ".join(texto_anonimizado)

# Cargar datos preprocesados
df_preprocesado = pd.read_excel("MOSTRA_1_preprocesado.xlsx")

# Aplicar anonimización
df_preprocesado["body_anonimizado"] = df_preprocesado["body_preprocesado"].apply(anonimizar_texto)

# Guardar anonimizado
df_preprocesado.to_excel("MOSTRA_1_anonimizado.xlsx", index=False)

# Guardar cambios en nombres
cambios_df = pd.DataFrame(cambios, columns=["Original", "Anonimizado", "Palabra_Anterior", "Palabra_Posterior"])
cambios_df.to_excel("cambios_anonimizacion.xlsx", index=False)

print("Anonimización completada. Ejecutando NER para validación...")

### 🔍 Integración con spaCy NER
def extraer_entidades(texto):
    """Extrae entidades nombradas de un texto usando spaCy."""
    doc = nlp(str(texto))
    return {ent.text for ent in doc.ents}  # Devolver conjunto de entidades únicas

# Aplicar NER en ambas versiones
df_preprocesado["entidades_original"] = df["body"].apply(extraer_entidades)
df_preprocesado["entidades_anonimizado"] = df_preprocesado["body_anonimizado"].apply(extraer_entidades)

# Comparar entidades y decidir si revertir al texto original
def restaurar_si_cambia(row):
    if row["entidades_original"] != row["entidades_anonimizado"]:
        return row["body"]  # Restaurar original si hay cambios
    return row["body_anonimizado"]  # Mantener anonimizado si no hay cambios

df_preprocesado["body_final"] = df_preprocesado.apply(restaurar_si_cambia, axis=1)

# Guardar resultado final
df_preprocesado.to_excel("MOSTRA_1_final.xlsx", index=False)

print("Proceso completo. Archivo guardado como 'MOSTRA_1_final.xlsx'")